In [1]:
import pandas as pd

sheets = pd.read_excel('База данных породного состава_2022-12-01.xlsx', sheet_name=None)
print(f'{len(sheets)} sheets: {" --- ".join(sorted(sheets.keys()))}')

18 sheets: Ассортимент растений --- Группы городов --- Группы растений --- Климатические зоны --- Лист13 --- Лист22 --- Лист23 --- Лист25 --- Матрица совместимости раститель --- НИР --- Параметрыхарактеристики --- Перечень деревьев --- РодВид --- Сущ. насажд. Выборгский район --- Сущ. насажд. Калининский р. --- Сущ. насажд. Колпинский р. --- Тип растительности --- мо №7


In [2]:
from datetime import date

names_mapping = {
    'Уст. к пересыханию': 'Устойчивость к пересыханию',
    'Уст. к подтоплению': 'Устойчивость к подтоплению',
    'Агресс-ть развития': 'Агрессивность развития',
    'Назначение, хар-р использ-я': 'Назначение, характер использования',
    'Ксерофит': 'Мало воды',
    'Мезофит': 'Средняя',
    'Гигрофит': 'Много воды',
    'Средне плодородные': 'Средне плодородная',
    'Бедные почвы': 'Бедная почва'
}

types_mapping = {}
for name in ('Устойчивость к переуплотнению', 'Устойчивость к засолению', 'Устойчивость к пересыханию',
        'Устойчивость к подтоплению', 'Газостойкость', 'Ветроустойчивость'):
    types_mapping[name] = 'limitation_factors'
for name in ('Сильнокислые (4)', 'Кислые (5)', 'Слабокислые (6)', 'Нейтральные (7)',
        'Слабощелочные (8)', 'Щелочные (9)', 'Сильнощелочные (10)'):
    types_mapping[name] = 'soil_acidity_types'
for name in ('Плодородная' , 'Средне плодородные' , 'Бедные почвы'):
    types_mapping[name] = 'soil_fertility_types'
for name in ('Песчаная', 'Супесчаная', 'Суглинистая', 'Глинистая', 'Каменистые', 'Щебнистые', 'Тяжёлая', 'Хорошо дренированная'):
    types_mapping[name] = 'soil_types'
for name in ('Полное освещение', 'Полутень', 'Тень'):
    types_mapping[name] = 'light_types'
for name in ('Мало воды', 'Средняя', 'Много воды'):
    types_mapping[name] = 'humidity_types'

df = sheets['Лист13'].copy()

df.iat[1, 0] = 'idx'
df.iat[1, 1] = 'Название'
df.iat[1, 2] = 'Латинское название'
try:
    df.at[1, 'Высота'] = 'Высота'
    df.at[1, 'Размер кроны'] = 'Размер кроны'
    df.at[1, 'Агресс-ть развития'] = 'Агресс-ть развития'
    df.at[1, 'Живучесть'] = 'Живучесть'
    df.at[1, 'Инвазивный вид'] = 'Инвазивный вид'
    df.at[1, 'Период цветения'] = 'Период цветения'
    df.at[1, 'Назначение, хар-р использ-я'] = 'Назначение, хар-р использ-я'
except Exception as ex:
    print(f'Exception at {ex}')
    df.iat[1, 14] = df.iat[0, 14]
    df.iat[1, 15] = df.iat[0, 15]
    df.iat[1, 49] = df.iat[0, 49]
    df.iat[1, 50] = df.iat[0, 50]
df.columns = df.iloc[1]
df = df.rename(names_mapping, axis=1).set_index('idx').iloc[3:]
df['Высота'] = df['Высота'].apply(lambda x: None if x == '-' else float(x) if isinstance(x, str) else float(f'{x.day}.{x.month}' if isinstance(x, date) else x))
df['Размер кроны'] = df['Размер кроны'].apply(lambda x: None if x == '-' else float(x) if isinstance(x, str) else float(f'{x.day}.{x.month}' if isinstance(x, date) else x))

df = df.dropna(subset='Латинское название').drop_duplicates(subset='Латинское название')
# df = df.drop_duplicates(subset='Латинское название').drop_duplicates()
# .append(df[df['Латинское название'].isna()]).drop_duplicates()
df
# 369 / 50
# 440 / 51

1,Название,Латинское название,USDA1,USDA2,USDA3,USDA4,USDA5,USDA6,USDA7,USDA8,...,Инвазивный вид,Полное освещение,Полутень,Тень,Мало воды,Средняя,Много воды,Период цветения,"Назначение, характер использования",NaN
idx,,,,,,,,,,,,,,,,,,,,,
1,Абрикос обыкновенный,Armeniaca vulgaris Lam.,NaN,NaN,NaN,NaN,1,1,1,1,...,0.0,1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
2,Абрикос сибирский,Prunus sibirica L.,NaN,1,1,1,1,1,1,1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Азалия понтийская,Azalea pontica,NaN,NaN,1,1,1,1,1,1,...,0.0,1,1,0,-1,1,NaN,NaN,NaN,NaN
4,Айва японская,Chaenomeles japonica (Thunb.) Spach,,NaN,NaN,NaN,1,1,1,1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Айлант высочайший,Ailanthus altissima (Mill.) Swingle,,NaN,NaN,1,1,1,1,1,...,1.0,1,1,NaN,0,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,Яблоня сливолистная,Malus prunifolia (Willd.) Borkh.,NaN,0,1,1,1,1,1,1,...,0.0,1,1,0,-1,1,0,NaN,NaN,NaN
517,Яблоня ягодная,Malus baccata (L.) Borkh.,NaN,0,1,1,1,1,1,1,...,0.0,1,1,-1,-1,1,-1,NaN,NaN,NaN
518,Ясень американский,Fraxinus americana L.,NaN,NaN,NaN,0,1,1,1,1,...,0.0,1,-1,-1,-1,1,-1,NaN,NaN,NaN


In [7]:
from tqdm.notebook import tqdm
from numpy import nan
import psycopg2

# conn = psycopg2.connect(host='localhost', port=5432, dbname='greendb', user='kanu', password='postgres', application_name='add_plants', connect_timeout=10)
conn = psycopg2.connect(host='10.32.1.107', port=5432, dbname='GreenDB', user='postgres', password='postgres', application_name='update_plants', connect_timeout=10)

In [8]:
import sys
with conn, conn.cursor() as cur, open('update_in_plants.txt', 'w', encoding='UTF-8') as f:
    for _, (name_ru, name_lat) in df[['Название', 'Латинское название']].iterrows():
        cur.execute('SELECT name_ru FROM plants WHERE name_latin = %s', (name_lat,))
        if (res := cur.fetchone()) is not None:
            if res[0] != name_ru:
                for output in (sys.stdout, f):
                    print(f'Updating in DB: {res[0]} -> {name_ru} (name_latin: {name_lat})', file=output)
                cur.execute('UPDATE plants SET name_ru = %s WHERE name_latin = %s', (name_ru, name_lat))
    if input("Commit? ").lower() not in ('y', '1'):
        conn.rollback()
        print('Aborted')
    else:
        print('Commited')

Updating in DB: Вейгела ранняя ранняя -> Вейгела ранняя (name_latin: Weigela praecox (Lemoine) L.H.Bailey)
Updating in DB: Слива мелкопильчатая -> Вишня мелкопильчатая (name_latin: Prunus serrulata Lindl.)
Updating in DB: Гайярдия остистая -> Гайлардия остистая (name_latin: Gaillardia aristata Pursh)
Updating in DB: Гвоздика гренобольская -> Гвоздика гренобльская (name_latin: Dianthus gratianopolitanus Vill.)
Updating in DB: Ель обыкновенная -> Ель европейская (name_latin: Picea abies (L.) H.Karst.)
Updating in DB: Лох смешиваемый / Лох смешиваемый -> Лох смешиваемый (name_latin: Elaeagnus commutata Bernh. ex Rydb.)
Updating in DB: Арония черноплодная -> Рябина обыкновенная (name_latin: Sorbus aucuparia L.)
Updating in DB: Черёмуха виргинская -> Черемуха виргинская (name_latin: Padus virginiana (L.) Mill.)
Updating in DB: Черёмуха Маака -> Черемуха Маака (name_latin: Padus maackii (Rupr.) Kom.)
Updating in DB: Черёмуха обыкновенная -> Черемуха обыкновенная (name_latin: Padus avium Mill

In [9]:
inserted_new = 0
inserted_factors = 0
updated_factors = 0

type_table_name = {
    'limitation_factors': ('plants_limitation_factors', 'limitation_factor_id', 'limitation_factors'),
    'soil_acidity_types': ('plants_soil_acidity_types', 'soil_acidity_type_id', 'soil_acidity_types'),
    'soil_fertility_types': ('plants_soil_fertility_types', 'soil_fertility_type_id', 'soil_fertility_types'),
    'soil_types': ('plants_soil_types', 'soil_type_id', 'soil_types'),
    'light_types': ('plants_light_types', 'light_type_id', 'light_types'),
    'humidity_types': ('plants_humidity_types', 'humidity_type_id', 'humidity_types')
}

with conn, conn.cursor() as cur:
    for idx, line in tqdm(df.replace({nan: None}).iterrows(), total=df.shape[0]):
        cur.execute('SELECT id FROM plants WHERE name_ru = %s', (line['Название'],))
        if (res := cur.fetchone()) is None:
            cur.execute('INSERT INTO plants (name_ru, name_latin, height_avg, crown_diameter, spread_aggressiveness_level, survivability_level, is_invasive) VALUES (%s, %s, %s, %s, %s, %s, %s) RETURNING id',
                (line['Название'], line['Латинское название'], line['Высота'], line['Размер кроны'], line['Агрессивность развития'], line['Живучесть'], bool(line['Инвазивный вид'])))
            res = cur.fetchone()
            inserted_new += 1
        plant_id = res[0]
        for name, value in line.items():
            if name not in types_mapping or value not in ('0', '1', 0, 1):
                continue
            value = value == '1' or value == 1
            if name.startswith('USDA'):
                cur.execute('SELECT value FROM climate_zones_growing'
                        ' WHERE is_stable = %s AND climate_zone_id = (SELECT id FROM climate_zones WHERE name = %s)',
                        (plant_id, name))
                if (res := cur.fetchone()) is None:
                    cur.execute('INSERT INTO climate_zones_growing (plant_id, climate_zone_id, is_stable)'
                            ' VALUES (%s, (SELECT id FROM climate_zones WHERE name = %s), %s)', (plant_id, name, value))
                    inserted_factors += 1
                elif res[0] != value:
                    cur.execute('UPDATE climate_zones_growing SET is_stable = %s'
                            ' WHERE plant_id = %s AND climate_zone_id = (SELECT id FROM climate_zones WHERE name = %s)',
                            (value, plant_id, name))
                    updated_factors += 1
                continue
            try:
                # match types_mapping[name]:
                # case 'humidity_types':
                table_values = type_table_name[types_mapping[name]]
                cur.execute(f'SELECT is_stable FROM {table_values[0]}'
                        f' WHERE plant_id = %s AND {table_values[1]} = (SELECT id FROM {table_values[2]} WHERE name = %s)',
                        (plant_id, name))
                if (res := cur.fetchone()) is None:
                    cur.execute(f'INSERT INTO {table_values[0]} (plant_id, {table_values[1]}, is_stable)'
                            f' VALUES (%s, (SELECT id FROM {table_values[2]} WHERE name = %s), %s)', (plant_id, name, value))
                    inserted_factors += 1
                elif res[0] != value:
                    cur.execute(f'UPDATE {table_values[0]} SET is_stable = %s'
                            f' WHERE plant_id = %s AND {table_values[1]} = (SELECT id FROM {table_values[2]} WHERE name = %s)',
                            (value, plant_id, name))
                    updated_factors += 1
            except Exception:
                print(f'{types_mapping[name]} - {name} : {value}')
                raise
print(f'Inserted {inserted_new} new plants')
print(f'Inserted {inserted_factors} new factors values')
print(f'{updated_factors} factors updated')

  0%|          | 0/515 [00:00<?, ?it/s]

Inserted 84 new plants
Inserted 1117 new factors values
14 factors updated


In [10]:
conn.close()